In [27]:
import numpy as np
import collections

wordToSupertag = dict()
sufToSupertag = collections.defaultdict(list)

with open('../../NLP_Resources/supertags.txt') as f:
    temp = [tuple(x.split()) for x in f]
    wordToSupertag = {word: tag for (word, tag) in temp}
    for word,tag in temp:
        sufToSupertag[word[-3:]].append(tag)

In [28]:
unigrams = collections.defaultdict(int)

with open('../../NLP_Resources/polish_corpora.txt') as f:
    for line in f:
        line = line.lower()
        words = line.split()
        for word in words:
            unigrams[word] += 1

In [29]:
import random
unigramsByTag = collections.defaultdict(list)
probabilitiesByTag = collections.defaultdict(list)

for key in unigrams.keys():
    tag = wordToSupertag.get(key, '')
    if tag == '':
        tag = random.choice(sufToSupertag.get(word[-3:], ["empty"]))
    if tag != 'empty':
        unigramsByTag[tag].append(key)
        probabilitiesByTag[tag].append(unigrams.get(key, 0))


In [30]:
testString = 'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'
testString = testString.lower()



In [31]:
def generate_unigrams(testString):
    result = ''
    for word in testString.split():
        tag = wordToSupertag.get(word, '')
        if tag == '':
            tag = random.choice(sufToSupertag.get(word[-3:], ["empty"]))
        probs = np.array(probabilitiesByTag[tag])
        s = np.sum(probs)
        index = np.random.choice(len(unigramsByTag[tag]), p = probs / s)
        result += unigramsByTag[tag][index]
        result += ' '
    return result.rstrip()

In [32]:
print(testString, generate_unigrams(testString), sep = '\n')

mały piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
wiosenny wacław zajął w niezbędnej stronie dokładnie ustaloną ofertę


In [33]:
def load_data(file):
    K = 5
    with open(file) as f:
        temp = 1
        res = [x.split() for x in f if int(x.split()[0]) >= K]
        return res
bigramData = load_data('../../NLP_Resources/poleval_2grams.txt')

In [34]:
bigramsByTag = collections.defaultdict(list)
bigramProbsByTag = collections.defaultdict(list)

for el in bigramData:
    value, word1, word2 = el
    tag1 = wordToSupertag.get(word1, '')
    if tag1 == '':
        tag1 = random.choice(sufToSupertag.get(word1[-3:], ["empty"]))
    tag2 = wordToSupertag.get(word2, '')
    if tag2 == '':
        tag2 = random.choice(sufToSupertag.get(word2[-3:], ["empty"]))
    if tag1 != 'empty' and tag2 != 'empty':
        bigramsByTag[tag1 + '#' + tag2].append((word1, word2))
        bigramProbsByTag[tag1+'#'+tag2].append(value)
    
        

In [55]:
def generate_bigrams(testString):
    data = testString.split()
    result = ''
    for i in range(len(data) - 1):
        w1 = data[i]
        w2 = data[i+1]
        tag1 = wordToSupertag.get(w1, '')
        if tag1 == '':
            tag1 = random.choice(sufToSupertag.get(w1[-3:], ["empty"]))
        tag2 = wordToSupertag.get(word, '')
        if tag2 == '':
            tag2 = random.choice(sufToSupertag.get(w2[-3:], ["empty"]))

        probs = np.array(bigramProbsByTag.get(tag1+'#'+tag2, [])).astype(np.int)

        if len(probs) != 0:
            s = np.sum(probs)
            index = np.random.choice(len(bigramsByTag[tag1+'#'+tag2]), p = probs / s)
            if i == len(data) - 2:
                result += ' '.join(bigramsByTag[tag1+'#'+tag2][index]) + ' '
            else:
                result += bigramsByTag[tag1+'#'+tag2][index][0] + ' '
        else:
            probs = np.array(probabilitiesByTag[tag1])
            s = np.sum(probs)
            index = np.random.choice(len(unigramsByTag[tag1]), p = probs / s)
            result += unigramsByTag[tag1][index]
            result += ' '
    return result
        

In [59]:
print(testString, generate_bigrams(testString), sep = '\n')

mały piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
jednoręki emeryt odegrał w niezależnej trasie powszechnie zgłoszoną propozycję 
